In [ ]:
import copy

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
from tqdm.auto import tqdm
from scipy.stats import linregress

In [ ]:
zec50_1000 = np.ones(1001) * np.nan
ecs = np.ones(1001) * np.nan
tcr = np.ones(1001) * np.nan
deltaco2conc50_1000 = np.ones(1001) * np.nan
f50_1000 = np.ones(1001) * np.nan

In [ ]:
scenarios = ['1pct-branch-1000PgC']

In [ ]:
conc_df = pd.read_csv('../data/1pctCO2_concentrations.csv', index_col=0)

In [ ]:
conc_df

In [ ]:
fair_params_df = pd.read_csv('../data/calibrated_constrained_parameters.csv', index_col=0)

In [ ]:
species = ['CO2', 'CH4', 'N2O']
conc_properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
emis_properties = {
    "CO2": {
        'type': 'co2',
        'input_mode': 'emissions',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "CH4": {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    },
    "N2O": {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False
    }
}

In [ ]:
# do one at a time
for idx, iconf in tqdm(enumerate(fair_params_df.index), total=1001):
    # 140 year 1pct CO2
    fc = FAIR()
    fc.define_time(1850, 1990, 1)
    fc.define_configs(list(fair_params_df.index[0:1]))
    fc.define_scenarios(scenarios)
    fc.define_species(species, conc_properties)
    fc.allocate()
    fc.concentration.loc[dict(specie='CH4')] = 808.2490285
    fc.concentration.loc[dict(specie='N2O')] = 273.021047
    fc.concentration.loc[dict(specie='CO2')] = conc_df.values[:,None]
    fc.fill_species_configs()
    fill(fc.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'c1':'c3'])
    fill(fc.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'kappa1':'kappa3'])
    fill(fc.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'epsilon'])
    fill(fc.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma'])
    fill(fc.climate_configs['stochastic_run'], False)
    fill(fc.species_configs['iirf_0'], fair_params_df.loc[iconf, 'r0'], specie='CO2')
    fill(fc.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'rA'], specie='CO2')
    fill(fc.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'rU'], specie='CO2')
    fill(fc.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'rT'], specie='CO2')
    fill(fc.species_configs["forcing_scale"], fair_params_df.loc[iconf, "scale CO2"], specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['baseline_concentration'], 273.021047, specie='N2O')
    fill(fc.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')
    initialise(fc.concentration, fc.species_configs['baseline_concentration'])
    initialise(fc.forcing, 0)
    initialise(fc.temperature, 0)
    initialise(fc.airborne_emissions, 0)
    initialise(fc.cumulative_emissions, 0)
    fc.run(progress=False)
    # pl.plot(fc.timepoints, fc.emissions[:,0,0,0]/3.664)
    # pl.ylim(0, 40)
    # fc.cumulative_emissions[:, 0, 0, 0]/3.664
    branch1000 = np.argmax(fc.cumulative_emissions[:,0,0,0].data/3.664>1000)
    # fc.cumulative_emissions[branch1000,0,0,0].data/3.664

    # re-run C-driven to branch and save out restart
    fc = FAIR()
    fc.define_time(1850, 1850+branch1000, 1)
    fc.define_scenarios(scenarios)
    fc.define_configs([iconf])
    fc.define_species(species, conc_properties)
    fc.allocate()
    fc.concentration.loc[dict(specie='CH4')] = 808.2490285
    fc.concentration.loc[dict(specie='N2O')] = 273.021047
    fc.concentration.loc[dict(specie='CO2')] = conc_df.values[:branch1000+1,None]
    fc.fill_species_configs()
    fill(fc.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'c1':'c3'])
    fill(fc.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'kappa1':'kappa3'])
    fill(fc.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'epsilon'])
    fill(fc.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma'])
    fill(fc.climate_configs['stochastic_run'], False)
    fill(fc.species_configs['iirf_0'], fair_params_df.loc[iconf, 'r0'], specie='CO2')
    fill(fc.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'rA'], specie='CO2')
    fill(fc.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'rU'], specie='CO2')
    fill(fc.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'rT'], specie='CO2')
    fill(fc.species_configs["forcing_scale"], fair_params_df.loc[iconf, "scale CO2"], specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['baseline_concentration'], 273.021047, specie='N2O')
    fill(fc.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
    fill(fc.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
    fill(fc.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')
    initialise(fc.concentration, fc.species_configs['baseline_concentration'])
    initialise(fc.forcing, 0)
    initialise(fc.temperature, 0)
    initialise(fc.airborne_emissions, 0)
    initialise(fc.cumulative_emissions, 0)
    fc.run(progress=False)
    # fc.airborne_emissions[-1,0,0,0].data/3.664
    # fc.airborne_fraction[-1,0,0,0].data
    # fc.gas_partitions[0,0,0,:].data

    # Switch to emissions driven with zero emissions and run for 100 years.
    fe = FAIR()
    fe.define_time(1850+branch1000, 1850+branch1000+100, 1)
    fe.define_scenarios(['1pct-branch-1000PgC'])
    fe.define_configs([iconf])
    fe.define_species(species, emis_properties)
    fe.allocate()
    fe.concentration.loc[dict(specie='CH4')] = 808.2490285
    fe.concentration.loc[dict(specie='N2O')] = 273.021047
    fe.emissions.loc[dict(specie='CO2')] = 0
    fe.fill_species_configs()
    fill(fe.climate_configs['ocean_heat_capacity'], fair_params_df.loc[iconf,'c1':'c3'])
    fill(fe.climate_configs['ocean_heat_transfer'], fair_params_df.loc[iconf,'kappa1':'kappa3'])
    fill(fe.climate_configs['deep_ocean_efficacy'], fair_params_df.loc[iconf,'epsilon'])
    fill(fe.climate_configs['gamma_autocorrelation'], fair_params_df.loc[iconf,'gamma'])
    fill(fe.climate_configs['stochastic_run'], False)
    fill(fe.species_configs['iirf_0'], fair_params_df.loc[iconf, 'r0'], specie='CO2')
    fill(fe.species_configs['iirf_airborne'], fair_params_df.loc[iconf, 'rA'], specie='CO2')
    fill(fe.species_configs['iirf_uptake'], fair_params_df.loc[iconf, 'rU'], specie='CO2')
    fill(fe.species_configs['iirf_temperature'], fair_params_df.loc[iconf, 'rT'], specie='CO2')
    fill(fe.species_configs["forcing_scale"], fair_params_df.loc[iconf, "scale CO2"], specie='CO2')
    fill(fe.species_configs['baseline_concentration'], 284.3169988, specie='CO2')
    fill(fe.species_configs['baseline_concentration'], 808.2490285, specie='CH4')
    fill(fe.species_configs['baseline_concentration'], 273.021047, specie='N2O')
    fill(fe.species_configs['forcing_reference_concentration'], 284.3169988, specie='CO2')
    fill(fe.species_configs['forcing_reference_concentration'], 808.2490285, specie='CH4')
    fill(fe.species_configs['forcing_reference_concentration'], 273.021047, specie='N2O')
    initialise(fe.concentration, fc.concentration[-1, ...])
    initialise(fe.forcing, fc.forcing[-1, ...])
    initialise(fe.temperature, fc.temperature[-1, ...])
    initialise(fe.airborne_emissions, fc.airborne_emissions[-1, ...])
    initialise(fe.cumulative_emissions, fc.cumulative_emissions[-1, ...])
    initialise(fe.alpha_lifetime, fc.alpha_lifetime[-1, ...])
    fe.gas_partitions=fc.gas_partitions.copy()
    fe.run(progress=False)
    
    zec50_1000[idx] = fe.temperature[50,0,0,0] - fe.temperature[0,0,0,0]
    ecs[idx] = fe.ebms.ecs.data[0]
    tcr[idx] = fe.ebms.tcr.data[0]
    deltaco2conc50_1000[idx] = fe.concentration[50,0,0,0] - fe.concentration[0,0,0,0]
    f50_1000[idx] = fe.forcing[50,0,0,0] - fe.forcing[0,0,0,0]

In [ ]:
pl.plot(fc.timepoints, fc.emissions[:, 0, 0, 0])
pl.plot(fe.timepoints, fe.emissions[:, 0, 0, 0])

In [ ]:
pl.plot(fc.timebounds, fc.concentration[:, 0, 0, 0])
pl.plot(fe.timebounds, fe.concentration[:, 0, 0, 0])

In [ ]:
pl.plot(fc.timebounds, fc.forcing[:, 0, 0, 0])
pl.plot(fe.timebounds, fe.forcing[:, 0, 0, 0])

In [ ]:
pl.plot(fc.timebounds, fc.temperature[:, 0, 0, 0])
pl.plot(fe.timebounds, fe.temperature[:, 0, 0, 0])

In [ ]:
pl.plot(fc.timebounds, fc.cumulative_emissions[:, 0, 0, 0])
pl.plot(fe.timebounds, fe.cumulative_emissions[:, 0, 0, 0])

In [ ]:
pl.plot(fc.timebounds, fc.airborne_fraction[:, 0, 0, 0])
pl.plot(fe.timebounds, fe.airborne_fraction[:, 0, 0, 0])

In [ ]:
df = pd.DataFrame(
    np.array([
        zec50_1000,
        ecs,
        tcr,
        deltaco2conc50_1000,
        f50_1000
    ]).T,
    index=fair_params_df.index, 
    columns=['zec50_1000', 'ecs', 'tcr', 'deltaco2conc50_1000', 'f50_1000']
)

In [ ]:
df

In [ ]:
df.to_csv('../data/zec_esm-cliff_ecs_tcr_co2.csv')